### This notebook uses Keith's diagnostic bash script to run ILAMB (without having to CMORize data first), generates paths, and submits the ilamb run script

###### To Do

- rosie_diagnostics works. 
- on jupyter. 

- look at output locally
- get output onto website
- make this script more generic
- comparision bash scripts are in 
-/glade/work/rfisher/diagnostics_ilamb_standalone/runs/clm5_sp



In [1]:
# Import all normal libraries. Likely some of these aren't needed
import os
import netCDF4 as nc4
import sys
import shutil
import numpy as np
import warnings
warnings.filterwarnings('ignore')
import xarray as xr
from matplotlib import pyplot as plt
#import datetime

##### Make function to create directories.

In [2]:
def make_directory(fileroot):   
    if(os.path.isdir(fileroot)):
        print('dir exists:'+fileroot)
    else:
        os.mkdir(fileroot)
        print('made: '+fileroot)

##### Make path names

In [3]:
#paths
USER='rfisher'

# where are the ILAMB data?
ilamb_path   ='/glade/work/rfisher/ilamb/ILAMB/MODELS/'

#ILAMB config file
config_file='ilamb_CLMFATES_SP.cfg'
config_path='config_files/'+config_file

#model setup path
model_setup_file='model_setup_fatesrtm.txt'
model_setup_path = 'model_setup_files/'+ model_setup_file

#ILAMB regions file
ilamb_regions='region_files/LandRegions.nc'

#### Keith's setup
list_models='RTM_ens3_case_1' 'RTM_ens3_case_2' 'RTM_ens3_case_3'
print('model config file='+ config_path)
print('model setup path='+model_setup_path)
print('ilamb_path='+ilamb_path)

model config file=config_files/ilamb_CLMFATES_SP.cfg
model setup path=model_setup_files/model_setup_fatesrtm.txt
ilamb_path=/glade/work/rfisher/ilamb/ILAMB/MODELS/


### ilamb diagnostic bash script


In [7]:
%%bash -s "$config_path" "$model_setup_path" "$ilamb_path" "$ilamb_regions"

config_path=$1  
model_setup_path=$2
ilamb_path=$3
ilamb_regions=$4

#PBS -N ilamb_diagnostics*
#PBS -q regular
#PBS -l select=4:ncpus=2:mpiprocs=2:mem=109GB
#PBS -l walltime=12:00:00
#PBS -A P93300041
#PBS -j oe
#PBS -k eod
#PBS -m abe
#PBS -M oleson@ucar.edu

echo $config_path
echo $model_setup_path

export BUILD_DIRNAME fates_rtm

#conda init bash
source /glade/work/oleson/miniconda3/etc/profile.d/conda.sh

export MPI_UNBUFFERED_STDIO=true
export TMPDIR=$TMPDIR
mkdir -p $TMPDIR

conda activate ilamb
export MPLBACKEND=Agg
export ILAMB_ROOT=/glade/p/cesm/lmwg_dev/oleson/ILAMB/ILAMB_all

echo "submitting ilamb"
#rosie modifications
mpiexec_mpt dplace -s 1 ilamb-run --config $config_path \
--model_root $ilamb_path \
--models  RTM_ens4_case_1 RTM_ens3_case_1 RTM_ens3_case_10 \
--build_dir ilamb_dir/$BUILD_DIRNAME/_build \
--define_regions $ilamb_regions \
--model_setup $model_setup_path \
--filter  .clm2.h0. \
--confrontation Albedo/CERES Evapotranspiration/MODIS GrossPrimaryProductivity/GBAF
#>> ${log_filename} 2>&1


config_files/ilamb_CLMFATES_SP.cfg
model_setup_files/model_setup_fatesrtm.txt
submitting ilamb

Setting up model results from model_setup_files/model_setup_fatesrtm.txt

                                  RTM_ens4_case_1
                                  RTM_ens3_case_1
                                 RTM_ens3_case_10

Parsing config file config_files/ilamb_CLMFATES_SP.cfg...

                 GrossPrimaryProductivity/Fluxnet Initialized
                    GrossPrimaryProductivity/GBAF Initialized
                         Evapotranspiration/GLEAM Initialized
                         Evapotranspiration/MODIS Initialized
                         EvaporativeFraction/GBAF Initialized
                               LatentHeat/Fluxnet Initialized
                                  LatentHeat/GBAF Initialized
                                       Runoff/Dai Initialized
                             SensibleHeat/Fluxnet Initialized
                                SensibleHeat/GBAF Initialized
